## Leaky integrated firing neuron model  
## Few feautures :
### Spatial network, no memory term but exponential decay of cumulative dose

This is a minor change of the Dodds and Watts model  
Implement adaptive rewiring and generate statistics

In [1]:
#importing the required libraries

import networkx as nx
import matplotlib.pyplot as plt #for plotting
import numpy as np #for using arrays and vectorising the code wherever possible
import scipy
import random
# from numba import jit #numba precomplier to make the code faster
import pandas as pd
import copy
import warnings
warnings.filterwarnings("ignore")

# Variables used in the simulation

In [2]:
N = 100
gamma = 1 #decay of cumulative dose per unit time (second)
T = 100 #total number of seconds
dt = 0.05 #seconds
'''number of simulation steps is T/dt'''

'''Dodds watts parameters'''
p = 1 #rate of dose transfer (per unit time (second))
p = p*dt #rate of dose transfer per unit simulation step
r = 1/dt
r = r*dt
rho = 1/dt
rho = rho*dt
# r = 0.5
# r = r*dt

d_star = 1
# dose_quantity = 1
D_ini = 3

'''network parameters'''
circ_rad = 100 #for a network in which each node is connected to other nodes withing a circle of radius circ_rad
nn = 7 #the number of nearest neighbours(i.e. k) in knn
average_degree = 6
total_edges = N*2

'''rewiring parameters'''
# p_rew = 5 #rate of rewiring(per unit time(second))
# p_rew = p_rew*dt

'rewiring parameters'

# Creating the network. 
### The network outputs coordinates, distance matrix, edge_list and adjacency matrix

In [3]:
'''
obtains the number of nodes N
generates N points
calculates euclidean distance between each pair of points
returns the coordinates of the points and the distance matrix which is N*N
'''
def calc_dist(N) :
    coords = []
    for counter in range(N) :
        coords.append((N*np.random.random(),N*np.random.random()))

    dist_mat = scipy.spatial.distance.cdist(coords,coords)
    return dist_mat,coords

### 1. metric network

In [4]:
'''network = connect to points withing a circle of radius'''
#creates a link between points/nodes which satisfies the conditions of the network
def coupling(dist_mat,circ_rad) :
    edge_list = []
    edge_mask = (dist_mat<circ_rad) & (dist_mat>0)
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(edge_mask))))
#     print(type(np.ndarray.tolist(np.transpose(np.where(edge_mask)))))
    return edge_list

### 2. K nearest neighbour network

In [5]:
'''network = knn'''
def knn(dist_mat,nn,N) :
    near_neigh = np.argsort(dist_mat)
    selec_near_neigh = np.zeros((N,nn))
    selec_near_neigh = near_neigh[:,0:nn+1]

    edge_list = []
    for i in range(N) :
        for j in range(1,nn+1) :
            link = [i,selec_near_neigh[i,j]]
            edge_list.append(link)

    return edge_list

### 3. Random Network : GNP type

In [6]:
'''random network'''
def rand_network(average_degree,N) :
    z1 = np.random.uniform(size = (N,N))
    E,F = np.meshgrid(np.arange(0,N),np.arange(0,N))
    mask = ((average_degree/N) > z1) & (E!=F)
    adjacency_matrix = np.int64(np.zeros(shape=(N,N)))
    adjacency_matrix[mask] = np.int64(1)
    edge_list = []
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(adjacency_matrix==1))))
    return edge_list,adjacency_matrix

### 4. Random Network : GNM type

In [7]:
def rand_net_gnm(total_edges,N) :
    the_graph = nx.gnm_random_graph(N, total_edges,directed=True)
    adjacency_matrix = nx.adjacency_matrix(the_graph)
    adjacency_matrix = np.asarray(adjacency_matrix.todense())
    np.fill_diagonal(adjacency_matrix,0)
    edge_list = []
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(adjacency_matrix==1))))
    return edge_list

# Function defintions required for the main part of the script

In [8]:
# # @jit(nopython=True)
# '''infecting the left part of space.'''
# def left_part_infec(N) :
#     x_coord = []
#     y_coord = []
#     for j in range(len(coords)) :
#         x_coord.append(coords[j][0])
#         y_coord.append(coords[j][1])
#     x_coord = np.asarray(x_coord)
#     y_coord = np.asarray(y_coord)
#     points = np.asarray(np.where(x_coord<(N/5))) 
#     indi_state[points.T] = 2

#     return indi_state

In [9]:
'''infecting connected nodes'''
def InfectNetworkNeighbors(net,seed_node,init_infected_nodes):
     # if in bulk find one node randomly, and infect its neighbours
    infected_nodes = set()
    candidate_nodes = set()
    explored_nodes = set()

    #pick the seed node
    infected_nodes.add(seed_node)
    explored_nodes.add(seed_node)

    curr_node=seed_node

    #add its neighbors to the list of candidates
    for n in net.neighbors(curr_node):
        candidate_nodes.add(int(n))
    #print( curr_node)
    #print( candidate_nodes)

    #while we need to select more nodes...
    while len(infected_nodes) < init_infected_nodes:

        #if there are candidate nodes, select one of them
        if(len(candidate_nodes) > 0):
            new_node = np.random.choice(list(candidate_nodes),1)[0]
            infected_nodes.add(new_node)
            candidate_nodes.remove(new_node)

        elif len(infected_nodes - explored_nodes) > 0:
            curr_node = np.random.choice(list(infected_nodes -
            explored_nodes),1)[0]
            explored_nodes.add(curr_node)
            for n in set(net.neighbors(curr_node)) - infected_nodes:
                candidate_nodes.add(n)

        else:
#             print('Initial node infection step failed')
            return None
    return infected_nodes

In [10]:
''' function to transfer doses'''
# @jit(nopython=True)
def dose(adj_mat,p,partner_state_prev,d) :

    z1 = np.random.uniform(size=(len(adj_mat),len(adj_mat[0])))
    
    dose_transfer_mask = p > z1
    mod_adj_mat = np.multiply(adj_mat,z1) #modifying adjacency matrix to include the proability of dose transfer
    dose_mask = (p>mod_adj_mat) & (adj_mat!=0) & (partner_state_prev == 2)
    d[dose_mask] = dose_quantity #whenever it is proabable, dose transfer occurs
    
    return d

In [11]:
'''rewiring function which returns a new adjacency matrix'''

def rew(p_rew,adjacency_matrix,indi_state_for_rew_prev) :
    
    z1_rew = np.random.uniform(size=(len(adj_mat),len(adj_mat[0])))
    
    rew_mask_plus = (p_rew > z1_rew)  & (indi_state_for_rew_prev == 1)
    #create links. Dont break any.
    adjacency_matrix[rew_mask_plus] = 1
    rew_mask_minus = (p_rew > z1_rew) & (indi_state_for_rew_prev == 2)
    #break links. Dont create any.
    adjacency_matrix[rew_mask_minus] = 0
    np.fill_diagonal(adjacency_matrix,0)
    
    return adjacency_matrix

In [12]:
'''decay function which breaks exisiting links of a node without depending on other nodes'''

def decay(lamb_da,adjacency_matrix,OD_mesh2) :
    z1_decay = np.random.uniform(size=(len(adj_mat),len(adj_mat[0])))
    decay_mask = (lamb_da*OD_mesh2 > z1_decay) 
    adjacency_matrix[decay_mask] = 0
    
    return adjacency_matrix

In [13]:
'''function to update cumulative doses'''
def cumu_dose(d,D_prev,gamma) :
    I = d.sum(axis=1).reshape(N,1)
#     I = 0
    D = (D_prev - (gamma*D_prev*dt)) + I
    return D

In [14]:
'''function to update the states of the nodes of the system'''
def upd_indi_state(D,d_star,indi_state_prev) :
    
    z2 = np.random.uniform(size=(N,1))
    z3 = np.random.uniform(size=(N,1))
    
    indi_state = indi_state_prev
    
    indi_state_mask1 = (D>=d_star) & (indi_state_prev==1)
    indi_state[indi_state_mask1] = 2
    
    indi_state_mask21 = (D<d_star) & (indi_state_prev==2) & (r>=z2) & (rho>=z3)
    indi_state[indi_state_mask21] = 1
    
    indi_state_mask22 = (D<d_star) & (indi_state_prev==2) & (r>=z2) & (rho<z3)
    indi_state[indi_state_mask22] = 3
    
    indi_state_mask23 = (D<d_star) & (indi_state_prev==2) & (r<z2)
    indi_state[indi_state_mask23] = 2
    
    return indi_state

In [15]:
'''function to get/identify the infected nodes and the suceptible nodes'''
def states(indi_state) :
    infec_indi = []
    suscep_indi = []
    infec_indi = np.transpose(np.where(indi_state==2))
    suscep_indi = np.transpose(np.where(indi_state==1))
    return infec_indi,suscep_indi

In [16]:
def networkx_graph(coords,edge_list) :
    G = nx.DiGraph()
    pos = {(i): (coords[i][0],coords[i][1]) for i in range(N)}
    G.add_nodes_from(pos.keys())
    G.add_edges_from(edge_list)
    return G

In [17]:
def edge_from_adj_mat(adj_mat_list,q):
    edge_list = []
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(adj_mat_list[q]==1))))
    return edge_list

# Choose which network you want in the program
### Coupling network of knn network?

In [18]:
'''returns coordinates, distance matrix, edge_list and the adjacency matrix'''

dist_mat,coords = calc_dist(N) #node placement

# edge_list = coupling(dist_mat,circ_rad)
# edge_list = knn(dist_mat,nn,N)
# edge_list = rand_network(average_degree,N)
edge_list = rand_net_gnm(total_edges,N)

org_adj_mat = np.int64(np.zeros((N,N)))
for i in range(len(edge_list)):
    org_adj_mat[edge_list[i][0],edge_list[i][1]] = np.int64(1)

# The main part of the script

In [19]:
#infected state time series data frame
#columns indicate the time steps
timeseries_infec_frac = pd.DataFrame()

#in degree as columns and rows as nodes
timeseries_in_degree = pd.DataFrame() 

#out degree as columns and rows as nodes
timeseries_out_degree = pd.DataFrame()

#strongly connected components as columns
timeseries_connec_comps = pd.DataFrame()

In [20]:
p_rew_vals = [0.01,0.1,1]
start_vals = [1,10]
dose_quantity_vals = [0.1,2]

In [21]:
for sim in range(70,70+50) :
    '''returns coordinates, distance matrix, edge_list and the adjacency matrix'''
    dist_mat,coords = calc_dist(N) #node placement
    edge_list = rand_net_gnm(total_edges,N)
    org_adj_mat = np.int64(np.zeros((N,N)))
    for i in range(len(edge_list)):
        org_adj_mat[edge_list[i][0],edge_list[i][1]] = np.int64(1)
        
    df_row = 0
    for p_rew in p_rew_vals : #rate of rewiring (per unit time (second))
        lamb_da = p_rew/10
        lamb_da = lamb_da*dt
        p_rew = p_rew*dt
        for start in start_vals : #rate of dose transfer (per unit time (second))
            for dose_quantity in dose_quantity_vals : #number of nodes to infect initially
                print('(sim,lamb_da,p_rew,start,dose_quantity) = ', (sim,lamb_da/dt,p_rew/dt,start,dose_quantity))

                '''clearing exsiting data frames and creating new ones '''
                timeseries_infec_frac = pd.DataFrame()
                timeseries_in_degree = pd.DataFrame()
                timeseries_out_degree = pd.DataFrame()
                timeseries_connec_comps = pd.DataFrame()

                q = 0 #to generate new edge_list from the new adjacency matrix 
                t = np.arange(0,T,dt)

    #             numpy_array = np.zeros((N,len(t)),dtype = np.int64) #stores the states of the nodes. Rows are nodes and columns are time steps
                D_array = np.zeros((N,len(t))) #stores the cumulative doses. Rows are nodes and columns are time steps

                adj_mat_list = []
                adj_mat = copy.deepcopy(org_adj_mat)
                adj_mat_new = copy.deepcopy(adj_mat)
                adj_mat_list.append(adj_mat_new) #list of arrays which shows the time series of the adjacency matrix

                #coords remain the same. Get new edge_list from the latest adjacency matrix
                edge_list = edge_from_adj_mat(adj_mat_list,q)
                G = networkx_graph(coords,edge_list) #networkx graph

                '''from G, obtain the in degree and the out degree'''
                timeseries_in_degree.loc[:,0] = np.asarray([val for (node, val) in G.in_degree()])
                timeseries_out_degree.loc[:,0] = np.asarray([val for (node, val) in G.out_degree()])

                '''strongly and weekely connected components'''
                timeseries_connec_comps.loc[0,0] = nx.number_connected_components(G.to_undirected())

                '''choosing initially infected nodes'''
                indi_state = np.random.randint(1,2,size=(N,1))
                infected_nodes = None
                while infected_nodes is None : #infecting 'start' number of network neighbours
                    infected_nodes = InfectNetworkNeighbors(G,np.random.randint(N),start)
                print(infected_nodes)
                infected_nodes = np.asarray(list(infected_nodes)).reshape(len(infected_nodes),1)
                indi_state[infected_nodes[:,0],0] = 2

                A,B = np.meshgrid(indi_state,indi_state) 
                partner_state = A
                indi_state_for_rew = B

    #             numpy_array [:,0] = indi_state[:,0]

                indi_state_prev = indi_state
                partner_state_prev = partner_state
                indi_state_for_rew_prev = indi_state_for_rew

                d = np.zeros((len(adj_mat),len(adj_mat[0])))

                D = np.zeros((N,1))
                D[np.where(indi_state==2)] = D_ini
                D_array[:,0] = D[:,0]
                D_prev = D

                infec_frac = np.count_nonzero(indi_state == 2)/N
                timeseries_infec_frac.loc[df_row,0] = infec_frac
                '''the main part of the simulation'''
                counter = 0
                df_col = 1
                for t in np.arange(dt,T,dt) :
                    q = q + 1 
                    counter = counter + 1
                    infec_indi = []
                    suscep_indi = []
                    d = np.zeros((len(adj_mat),len(adj_mat[0])))
                    d = dose(adj_mat,p,partner_state_prev,d)

                    OD_array = np.asarray([val for (node, val) in G.out_degree()])
                    OD_mesh1,OD_mesh2 = np.meshgrid(OD_array,OD_array)
                    adj_mat = decay(lamb_da,adj_mat,OD_mesh2)
                    adj_mat = rew(p_rew,adj_mat,indi_state_for_rew_prev)
                    adj_mat_new = copy.deepcopy(adj_mat)
                    adj_mat_list.append(adj_mat_new)

                    #coords remain the same. Get new edge_list from the latest adjacency matrix
                    edge_list = edge_from_adj_mat(adj_mat_list,q)
                    G = networkx_graph(coords,edge_list) #networkx graph

                    '''from G, obtain the in degree and the out degree'''
                    timeseries_in_degree.loc[:,df_col] = np.asarray([val for (node, val) in G.in_degree()])
                    timeseries_out_degree.loc[:,df_col] = np.asarray([val for (node, val) in G.out_degree()])

                    '''strongly and weekely connected components'''
                    timeseries_connec_comps.loc[df_row,df_col] = nx.number_connected_components(G.to_undirected())

                    D = cumu_dose(d,D_prev,gamma)
                    D_array[:,counter] = D[:,0]

                    indi_state = upd_indi_state(D,d_star,indi_state_prev)
                    infec_frac = np.count_nonzero(indi_state == 2)/N
                    timeseries_infec_frac.loc[df_row,df_col] = infec_frac

    #                 numpy_array[:,counter] = indi_state[:,0]
                    A,B = np.meshgrid(indi_state,indi_state)

                    infec_indi, suscep_indi = states(indi_state)

                    partner_state = A
                    indi_state_for_rew = B
                    indi_state_prev = indi_state
                    partner_state_prev = partner_state
                    indi_state_for_rew_prev = indi_state_for_rew

                    D_prev = D
                    D = np.zeros((N,1))
                    df_col = df_col + 1 

    #             df_row = df_row + 1 #going to the next row of the df to store timeseries of next (p,start)

                filename_infec_frac = 'infec_frac.h5'
                filename_in_deg = 'in_deg.h5'
                filename_out_deg = 'out_deg.h5'
                filename_connec_comps = 'connec_comps.h5'
                p_rew_val = '%g'%(p_rew/dt)
                key_val = 'sim_'+str(sim)+'_p_rew_'+str(p_rew_val)+'_start_'+str(start)+'_dq_'+str(dose_quantity)
                print(key_val)
                timeseries_infec_frac.to_hdf(filename_infec_frac, key = key_val, mode='a')
                timeseries_in_degree.to_hdf(filename_in_deg, key = key_val, mode='a')
                timeseries_out_degree.to_hdf(filename_out_deg, key = key_val, mode='a')
                timeseries_connec_comps.to_hdf(filename_connec_comps, key = key_val, mode='a')
                df_row = 0

(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.001, 0.01, 1, 0.1)
{35}
sim_70_p_rew_0.01_start_1_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.001, 0.01, 1, 2)
{56}
sim_70_p_rew_0.01_start_1_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.001, 0.01, 10, 0.1)
{36, 72, 74, 77, 46, 14, 84, 90, 91, 31}
sim_70_p_rew_0.01_start_10_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.001, 0.01, 10, 2)
{32, 38, 72, 11, 13, 83, 51, 52, 54, 56}
sim_70_p_rew_0.01_start_10_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.01, 0.10000000000000002, 1, 0.1)
{60}
sim_70_p_rew_0.1_start_1_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.01, 0.10000000000000002, 1, 2)
{31}
sim_70_p_rew_0.1_start_1_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.01, 0.10000000000000002, 10, 0.1)
{2, 36, 72, 74, 77, 46, 84, 53, 89, 91}
sim_70_p_rew_0.1_start_10_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.01, 0.10000000000000002, 10, 2)
{65, 38, 72, 73, 75, 76, 83, 

(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.001, 0.01, 10, 2)
{36, 69, 6, 18, 84, 53, 24, 57, 92, 62}
sim_75_p_rew_0.01_start_10_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.01, 0.10000000000000002, 1, 0.1)
{67}
sim_75_p_rew_0.1_start_1_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.01, 0.10000000000000002, 1, 2)
{66}
sim_75_p_rew_0.1_start_1_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.01, 0.10000000000000002, 10, 0.1)
{66, 35, 6, 7, 41, 43, 79, 85, 56, 89}
sim_75_p_rew_0.1_start_10_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.01, 0.10000000000000002, 10, 2)
{64, 66, 99, 38, 71, 12, 45, 47, 58, 93}
sim_75_p_rew_0.1_start_10_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.10000000000000002, 1.0, 1, 0.1)
{97}
sim_75_p_rew_1_start_1_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.10000000000000002, 1.0, 1, 2)
{96}
sim_75_p_rew_1_start_1_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.10000000000000002, 1.0, 10, 0.1)
{

sim_80_p_rew_0.1_start_1_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (80, 0.01, 0.10000000000000002, 10, 0.1)
{96, 76, 44, 78, 48, 85, 87, 55, 58, 59}
sim_80_p_rew_0.1_start_10_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (80, 0.01, 0.10000000000000002, 10, 2)
{96, 6, 73, 59, 10, 78, 16, 48, 83, 27}
sim_80_p_rew_0.1_start_10_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (80, 0.10000000000000002, 1.0, 1, 0.1)
{42}
sim_80_p_rew_1_start_1_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (80, 0.10000000000000002, 1.0, 1, 2)
{35}
sim_80_p_rew_1_start_1_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (80, 0.10000000000000002, 1.0, 10, 0.1)
{96, 66, 98, 10, 47, 48, 52, 53, 85, 58}
sim_80_p_rew_1_start_10_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (80, 0.10000000000000002, 1.0, 10, 2)
{99, 37, 70, 40, 42, 77, 14, 15, 54, 58}
sim_80_p_rew_1_start_10_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.001, 0.01, 1, 0.1)
{84}
sim_81_p_rew_0.01_start_1_dq_0.1
(sim,lamb_da,p_r

(sim,lamb_da,p_rew,start,dose_quantity) =  (85, 0.10000000000000002, 1.0, 1, 0.1)
{46}
sim_85_p_rew_1_start_1_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (85, 0.10000000000000002, 1.0, 1, 2)
{65}
sim_85_p_rew_1_start_1_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (85, 0.10000000000000002, 1.0, 10, 0.1)
{1, 3, 36, 73, 42, 85, 54, 30, 62, 95}
sim_85_p_rew_1_start_10_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (85, 0.10000000000000002, 1.0, 10, 2)
{5, 6, 39, 10, 43, 21, 22, 88, 89, 26}
sim_85_p_rew_1_start_10_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (86, 0.001, 0.01, 1, 0.1)
{78}
sim_86_p_rew_0.01_start_1_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (86, 0.001, 0.01, 1, 2)
{48}
sim_86_p_rew_0.01_start_1_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (86, 0.001, 0.01, 10, 0.1)
{66, 68, 5, 37, 9, 46, 47, 80, 22, 24}
sim_86_p_rew_0.01_start_10_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (86, 0.001, 0.01, 10, 2)
{98, 6, 39, 74, 50, 84, 86, 25, 92, 62}
sim_86_

sim_90_p_rew_1_start_10_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (90, 0.10000000000000002, 1.0, 10, 2)
{64, 32, 98, 99, 34, 8, 81, 85, 91, 92}
sim_90_p_rew_1_start_10_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (91, 0.001, 0.01, 1, 0.1)
{33}
sim_91_p_rew_0.01_start_1_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (91, 0.001, 0.01, 1, 2)
{8}
sim_91_p_rew_0.01_start_1_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (91, 0.001, 0.01, 10, 0.1)
{70, 79, 81, 18, 51, 19, 22, 24, 28, 63}
sim_91_p_rew_0.01_start_10_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (91, 0.001, 0.01, 10, 2)
{32, 98, 73, 76, 49, 50, 18, 89, 93, 94}
sim_91_p_rew_0.01_start_10_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (91, 0.01, 0.10000000000000002, 1, 0.1)
{67}
sim_91_p_rew_0.1_start_1_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (91, 0.01, 0.10000000000000002, 1, 2)
{45}
sim_91_p_rew_0.1_start_1_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (91, 0.01, 0.10000000000000002, 10, 0.1)
{26,

sim_96_p_rew_0.01_start_1_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (96, 0.001, 0.01, 10, 0.1)
{6, 72, 11, 79, 15, 20, 93, 57, 92, 61}
sim_96_p_rew_0.01_start_10_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (96, 0.001, 0.01, 10, 2)
{33, 65, 3, 68, 43, 79, 17, 21, 90, 59}
sim_96_p_rew_0.01_start_10_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (96, 0.01, 0.10000000000000002, 1, 0.1)
{42}
sim_96_p_rew_0.1_start_1_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (96, 0.01, 0.10000000000000002, 1, 2)
{72}
sim_96_p_rew_0.1_start_1_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (96, 0.01, 0.10000000000000002, 10, 0.1)
{6, 71, 72, 11, 46, 79, 20, 21, 92, 93}
sim_96_p_rew_0.1_start_10_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (96, 0.01, 0.10000000000000002, 10, 2)
{35, 8, 44, 77, 45, 82, 83, 51, 20, 85}
sim_96_p_rew_0.1_start_10_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (96, 0.10000000000000002, 1.0, 1, 0.1)
{88}
sim_96_p_rew_1_start_1_dq_0.1
(sim,lamb_da,p_rew,sta

(sim,lamb_da,p_rew,start,dose_quantity) =  (101, 0.01, 0.10000000000000002, 1, 0.1)
{17}
sim_101_p_rew_0.1_start_1_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (101, 0.01, 0.10000000000000002, 1, 2)
{89}
sim_101_p_rew_0.1_start_1_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (101, 0.01, 0.10000000000000002, 10, 0.1)
{2, 67, 40, 10, 77, 79, 83, 19, 56, 92}
sim_101_p_rew_0.1_start_10_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (101, 0.01, 0.10000000000000002, 10, 2)
{67, 79, 16, 82, 83, 50, 53, 22, 56, 92}
sim_101_p_rew_0.1_start_10_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (101, 0.10000000000000002, 1.0, 1, 0.1)
{76}
sim_101_p_rew_1_start_1_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (101, 0.10000000000000002, 1.0, 1, 2)
{68}
sim_101_p_rew_1_start_1_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (101, 0.10000000000000002, 1.0, 10, 0.1)
{67, 76, 14, 79, 82, 19, 52, 53, 92, 29}
sim_101_p_rew_1_start_10_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (101, 0.100

sim_106_p_rew_0.1_start_1_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (106, 0.01, 0.10000000000000002, 10, 0.1)
{32, 97, 66, 65, 70, 10, 43, 18, 56, 95}
sim_106_p_rew_0.1_start_10_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (106, 0.01, 0.10000000000000002, 10, 2)
{96, 64, 66, 34, 5, 43, 84, 21, 89, 26}
sim_106_p_rew_0.1_start_10_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (106, 0.10000000000000002, 1.0, 1, 0.1)
{96}
sim_106_p_rew_1_start_1_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (106, 0.10000000000000002, 1.0, 1, 2)
{92}
sim_106_p_rew_1_start_1_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (106, 0.10000000000000002, 1.0, 10, 0.1)
{32, 98, 37, 73, 47, 15, 17, 81, 23, 27}
sim_106_p_rew_1_start_10_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (106, 0.10000000000000002, 1.0, 10, 2)
{64, 98, 74, 12, 13, 48, 49, 29, 56, 93}
sim_106_p_rew_1_start_10_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (107, 0.001, 0.01, 1, 0.1)
{3}
sim_107_p_rew_0.01_start_1_dq_0.1
(s

sim_111_p_rew_0.1_start_10_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (111, 0.10000000000000002, 1.0, 1, 0.1)
{72}
sim_111_p_rew_1_start_1_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (111, 0.10000000000000002, 1.0, 1, 2)
{45}
sim_111_p_rew_1_start_1_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (111, 0.10000000000000002, 1.0, 10, 0.1)
{5, 6, 45, 78, 20, 85, 90, 59, 29, 30}
sim_111_p_rew_1_start_10_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (111, 0.10000000000000002, 1.0, 10, 2)
{33, 39, 72, 41, 10, 74, 14, 87, 55, 60}
sim_111_p_rew_1_start_10_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (112, 0.001, 0.01, 1, 0.1)
{13}
sim_112_p_rew_0.01_start_1_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (112, 0.001, 0.01, 1, 2)
{45}
sim_112_p_rew_0.01_start_1_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (112, 0.001, 0.01, 10, 0.1)
{0, 65, 33, 99, 35, 14, 24, 89, 25, 95}
sim_112_p_rew_0.01_start_10_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (112, 0.001, 0.01, 10, 

sim_116_p_rew_1_start_1_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (116, 0.10000000000000002, 1.0, 10, 0.1)
{69, 27, 42, 46, 14, 80, 50, 53, 88, 59}
sim_116_p_rew_1_start_10_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (116, 0.10000000000000002, 1.0, 10, 2)
{38, 74, 43, 45, 52, 86, 23, 89, 26, 27}
sim_116_p_rew_1_start_10_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (117, 0.001, 0.01, 1, 0.1)
{87}
sim_117_p_rew_0.01_start_1_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (117, 0.001, 0.01, 1, 2)
{38}
sim_117_p_rew_0.01_start_1_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (117, 0.001, 0.01, 10, 0.1)
{66, 3, 5, 76, 18, 82, 83, 85, 28, 95}
sim_117_p_rew_0.01_start_10_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantity) =  (117, 0.001, 0.01, 10, 2)
{99, 40, 74, 76, 81, 23, 88, 90, 60, 61}
sim_117_p_rew_0.01_start_10_dq_2
(sim,lamb_da,p_rew,start,dose_quantity) =  (117, 0.01, 0.10000000000000002, 1, 0.1)
{12}
sim_117_p_rew_0.1_start_1_dq_0.1
(sim,lamb_da,p_rew,start,dose_quantit

In [22]:
# filename = 'file_avdeg_'+str(average_degree)+'_ini_'+str(np.int64(X[0,0]*N))+'_'+str(np.int64(X[p_len-1,start_len-1]*N))+'.h5'
# key 
# timeseries_infec_frac.to_hdf(filename, key='timeseries_infec_frac', mode='w')

In [23]:
# import h5py
# f = h5py.File('in_deg.h5', 'r')
# [key for key in f.keys()]
# # pd.read_hdf('file_avdeg_6_ini_0_11_firstattemp.h5','p_rew_2')